# Test In Simulation

In [ ]:
from FR3Py.sim.mujoco import FR3Sim
robot = FR3Sim(interface_type='velocity')

In [ ]:
from ndcurves import SE3Curve
import pinocchio as pin
from FR3Py.robot.model import PinocchioModel
model = PinocchioModel()

In [ ]:
state = robot.getJointStates()
info = model.getInfo(state['q'], state['dq'])

In [ ]:
import numpy as np

FSM = {
        "PRE-GRASP": (np.array([[0.4], [0.4], [0.1], [0.04]]), 5.0),
        "GRASP": (np.array([[0.4], [0.4], [0.0125], [0.04]]), 2.0),
        "CLOSE-GRASP": (np.array([[0.4], [0.4], [0.013], [0.000]]), 2.0),
        "POST-GRASP": (np.array([[0.4], [0.4], [0.1], [0.000]]), 2.0),
        "PRE-PLACE": (np.array([[0.4], [-0.4], [0.1], [0.000]]), 5.0),
        "PLACE": (np.array([[0.4], [-0.4], [0.013], [0.000]]), 2.0),
        "OPEN-GRASP": (np.array([[0.4], [-0.4], [0.013], [0.04]]), 2.0),
        "POST-PLACE": (np.array([[0.4], [-0.4], [0.1], [0.04]]), 2.0),
        "RETURN": (np.array([[0.3], [0.0], [0.5], [0.00]]), 10.0),
    }

task_list = [
    "PRE-GRASP",
    "GRASP",
    "CLOSE-GRASP",
    "POST-GRASP",
    "PRE-PLACE",
    "PLACE",
    "OPEN-GRASP",
    "POST-PLACE",
    "RETURN",
]
task_id = 0
target, duration = FSM[task_list[task_id]]
t = 0.0

T_init = pin.SE3(info["R_EE"].copy(), info["P_EE"].copy())
R_end = np.diag([1.0, -1.0, -1.0])
p_end = target[:3, :]
T_end = pin.SE3(R_end, p_end)
t_init = t
t_end = t + duration
curve = SE3Curve(T_init, T_end, t_init, t_end)

In [ ]:
import numpy as np
from FR3Py.controllers.jacobianPseudoInv import WaypointController
controller = WaypointController(kp=1.5)

for i in range(40000):
    state = robot.getJointStates()
    info = model.getInfo(state['q'], state['dq'])
    T = pin.SE3(info["R_EE"].copy(), info["P_EE"].copy())
    t = np.clip(i * robot.dt, 0.0, t_end)
    T_des = curve(t)
    cmd = controller.compute(state['q'], state['dq'], T_cmd=T_des)
    robot.setCommands(cmd)
    robot.step()
    error = np.linalg.norm(target[:3, 0] - info["P_EE"])
    if t >= t_end and error <= 0.005:
        task_id = task_id + 1

        if task_id >= len(task_list):
            break

        target, duration = FSM[task_list[task_id]]

        T_init = pin.SE3(info["R_EE"].copy(), info["P_EE"].copy())
        R_end = np.diag([1.0, -1.0, -1.0])
        p_end = target[:3, 0]
        T_end = pin.SE3(R_end, p_end)
        t_init = t
        t_end = t + duration
        curve = SE3Curve(T_init, T_end, t_init, t_end)

# Test with Real Robot

In [ ]:
import sys 
from FR3Py.robot.interface import FR3Real
robot = FR3Real(robot_id='fr3')

If the interface the successfully exchanging data between the robot and the Python side, we should be able to read the state of the robot. Otherwise, we will get a None.

In [ ]:
robot.getJointStates()

In [ ]:
from ndcurves import SE3Curve
import pinocchio as pin
from FR3Py.robot.model import PinocchioModel
model = PinocchioModel()

In [ ]:
state = robot.getJointStates()
info = model.getInfo(state['q'], state['dq'])

In [ ]:
import numpy as np

FSM = {
        "PRE-GRASP": (np.array([[0.4], [0.4], [0.1], [0.04]]), 5.0),
        "GRASP": (np.array([[0.4], [0.4], [0.0125], [0.04]]), 2.0),
        "CLOSE-GRASP": (np.array([[0.4], [0.4], [0.013], [0.000]]), 2.0),
        "POST-GRASP": (np.array([[0.4], [0.4], [0.1], [0.000]]), 2.0),
        "PRE-PLACE": (np.array([[0.4], [-0.4], [0.1], [0.000]]), 5.0),
        "PLACE": (np.array([[0.4], [-0.4], [0.013], [0.000]]), 2.0),
        "OPEN-GRASP": (np.array([[0.4], [-0.4], [0.013], [0.04]]), 2.0),
        "POST-PLACE": (np.array([[0.4], [-0.4], [0.1], [0.04]]), 2.0),
        "RETURN": (np.array([[0.3], [0.0], [0.5], [0.00]]), 10.0),
    }

task_list = [
    "PRE-GRASP",
    "GRASP",
    "CLOSE-GRASP",
    "POST-GRASP",
    "PRE-PLACE",
    "PLACE",
    "OPEN-GRASP",
    "POST-PLACE",
    "RETURN",
]
task_id = 0
target, duration = FSM[task_list[task_id]]
t = 0.0

T_init = pin.SE3(info["R_EE"].copy(), info["P_EE"].copy())
R_end = np.diag([1.0, -1.0, -1.0])
p_end = target[:3, :]
T_end = pin.SE3(R_end, p_end)
t_init = t
t_end = t + duration
curve = SE3Curve(T_init, T_end, t_init, t_end)

In [ ]:
state

In [ ]:
import numpy as np
from FR3Py.controllers.jacobianPseudoInv import WaypointController
import time
controller = WaypointController(kp=1.5)
time.sleep(1)
dt = 0.01
for i in range(40000):
    state = robot.getJointStates()
    info = model.getInfo(state['q'], state['dq'])
    T = pin.SE3(info["R_EE"].copy(), info["P_EE"].copy())
    t = np.clip(i * dt, 0.0, t_end)
    T_des = curve(t)
    cmd = controller.compute(state['q'], state['dq'], T_cmd=T_des)
    robot.setCommands(cmd)
    time.sleep(dt)
    error = np.linalg.norm(target[:3, 0] - info["P_EE"])
    if t >= t_end and error <= 0.005:
        task_id = task_id + 1

        if task_id >= len(task_list):
            break

        target, duration = FSM[task_list[task_id]]

        T_init = pin.SE3(info["R_EE"].copy(), info["P_EE"].copy())
        R_end = np.diag([1.0, -1.0, -1.0])
        p_end = target[:3, 0]
        T_end = pin.SE3(R_end, p_end)
        t_init = t
        t_end = t + duration
        curve = SE3Curve(T_init, T_end, t_init, t_end)